In [ ]:
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@main#egg=nemo_toolkit[all]

# Converting ChatGPT Text to tsv

In [ ]:
import pandas as pd

outData = pd.DataFrame(columns = ["Written", "Spoken"])

with open("data/simple_text.txt", "r") as inputFile:
    line = inputFile.readline()

    while line:
        i = 0
        while line[i] != ".":
            i+= 1
        num = str(line[:i])

        i += 8
        written = line[i:len(line)-1]

        line = inputFile.readline()
        i=0
        while line[i] != 'S':
            i+= 1
        i += 8
        spoken = line[i:len(line)-1]

        outData.loc[len(outData)] = {"Written": written, "Spoken": spoken}
        inputFile.readline()
        line = inputFile.readline()

print(outData)

In [21]:
outData.to_csv("data/data.tsv", mode='a', index=False, sep="\t")

# Preprocessing Data

In [5]:
from nemo.collections.nlp.data.text_normalization_as_tagging.utils import spoken_preprocessing
import pandas as pd

df = pd.read_csv("data/data.tsv", sep="\t")

# print(df['Spoken'])

batch, all_preds = [], []
for i, line in enumerate(df["Spoken"]):
    s = spoken_preprocessing(line)  # this is the same input transformation as in corpus preparation
    batch.append(s.strip())
    print(s)
    print(type(s))

print(batch)

i have five cats
<class 'str'>
the temperature is thirty degrees celsius
<class 'str'>
she arrived at nine forty-five a m
<class 'str'>
the book is two point five inches thick
<class 'str'>
the meeting starts at eight o'clock a m sharp
<class 'str'>
he finished the race in one hour twenty minutes
<class 'str'>
the package weighs two point three kilograms
<class 'str'>
the movie starts at seven thirty p m
<class 'str'>
the distance between the two cities is one hundred fifty miles
<class 'str'>
the shirt costs nineteen dollars and ninety-nine cent
<class 'str'>
['i have five cats', 'the temperature is thirty degrees celsius', 'she arrived at nine forty-five a m', 'the book is two point five inches thick', "the meeting starts at eight o'clock a m sharp", 'he finished the race in one hour twenty minutes', 'the package weighs two point three kilograms', 'the movie starts at seven thirty p m', 'the distance between the two cities is one hundred fifty miles', 'the shirt costs nineteen dollar

In [11]:
from nemo.collections import nlp as nemo_nlp

thutmose = nemo_nlp.models.ThutmoseTaggerModel.from_pretrained("itn_en_thutmose_bert")

outputs = thutmose._infer(batch)

print(outputs)

[NeMo I 2023-06-26 19:47:42 cloud:68] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/itn_en_thutmose_bert/versions/1.9.0/files/itn_en_thutmose_bert.nemo to /home/sarwagya/.cache/torch/NeMo/NeMo_1.19.0rc0/itn_en_thutmose_bert/bf3a085f3b78525c3b3abe09bd0c62c8/itn_en_thutmose_bert.nemo
[NeMo I 2023-06-26 19:48:44 common:913] Instantiating model from pre-trained checkpoint
[NeMo I 2023-06-26 19:48:54 tokenizer_utils:130] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-base-uncased, vocab_file: /tmp/tmpsonnv61w/1742bd18a2874ba6b6cb1d643dabcb2b_vocab.txt, merges_files: None, special_tokens_dict: {}, and use_fast: False


Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.
[NeMo W 2023-06-26 19:48:57 modelPT:244] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.
[NeMo W 2023-06-26 19:48:57 nlp_overrides:249] Apex was not found. Please see the NeMo README for installation instructions: https://github.com/NVIDIA/apex
    Megatron-based models require Apex to function correctly.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassi

[NeMo I 2023-06-26 19:50:04 save_restore_connector:249] Model ThutmoseTaggerModel was successfully restored from /home/sarwagya/.cache/torch/NeMo/NeMo_1.19.0rc0/itn_en_thutmose_bert/bf3a085f3b78525c3b3abe09bd0c62c8/itn_en_thutmose_bert.nemo.
['i have five cats\ti have five cats\t<SELF> <SELF> <SELF> <SELF>\t<SELF> <SELF> <SELF> <SELF>\tPLAIN PLAIN PLAIN PLAIN', 'the temperature is 30 degrees celsius\tthe temperature is thirty degrees celsius\t<SELF> <SELF> <SELF> _30_ <SELF> <SELF>\t<SELF> <SELF> <SELF> _30_ <SELF> <SELF>\tPLAIN PLAIN PLAIN CARDINAL PLAIN PLAIN', 'she arrived at 09:40 am\tshe arrived at nine forty-five a m\t<SELF> <SELF> <SELF> _09: 40_ _am_ <DELETE>\t<SELF> <SELF> <SELF> _09: 40_ _am_ <DELETE>\tPLAIN PLAIN PLAIN TIME TIME TIME TIME', 'the book is 2.5 inches thick\tthe book is two point five inches thick\t<SELF> <SELF> <SELF> _2 . 5_ <SELF> <SELF>\t<SELF> <SELF> <SELF> _2 . 5_ <SELF> <SELF>\tPLAIN PLAIN PLAIN DECIMAL DECIMAL DECIMAL PLAIN PLAIN', "the meeting starts at

: 

In [10]:
with open("data/pretrained_nemo_output.tsv", "w") as file:
    for line in outputs:
        file.write(line)
        file.write("\n")